## An example workflow

In [1]:
import Poseidon
from Poseidon.model import *
from Poseidon.dem import *
from Poseidon.meteo import *
import Poseidon.model.mdf as mdf
from Poseidon.model.grid import *
from Poseidon.model.dep import *

### setup

In [2]:
#create a d3d model
b = d3d()

In [3]:
#define in a dictionary the properties e.g
dic={'lon0':0., # lat/lon window
     'lon1':10.,
     'lat0':28.,
     'lat1':38.,
     'tag':'test',
     'date':datetime.datetime(2013,3,1,0),
     'resolution':0.1}

In [4]:
b.set(**dic) # feed the dic to the model

create grid


In [5]:
#view model
#b.__dict__

### Define preferences 

In [6]:
#use this if the following fails
#import pkg_resources
#DATA_PATH = pkg_resources.resource_filename('Poseidon', 'misc/')

In [7]:
DATA_PATH = os.path.dirname(Poseidon.__file__)+'/misc/'

In [8]:
#read mdf file
inp,order = mdf.read(DATA_PATH+'default.mdf')

In [9]:
Tstart = 0
Tstop = 60
step = 60

In [10]:
  #define grid file
inp['Filcco']=b.tag+'.grd'
  
  #define enc file
inp['Filgrd']=b.tag+'.enc'
  
  #define dep file
inp['Fildep']=b.tag+'.dep'
  
  #define obs file
inp['Filsta']='' #b.tag+'.obs'
  
  
  # adjust ni,nj
inp['MNKmax']=[b.ni+1,b.nj+1,1]  # add one like ddb
  
  # adjust iteration date
inp['Itdate']=datetime.datetime.strftime(b.date,'%Y-%m-%d')
  
  #set time unit
inp['Tunit']='M'

  #adjust iteration start
inp['Tstart']=[Tstart]
  
  #adjust iteration stop
inp['Tstop']=[Tstop]
  
  #adjust time step
inp['Dt']=[1.]
  
  #adjust time for output
step=60
inp['Flmap']=[Tstart,step,Tstop]
inp['Flhis']=[Tstart,1,Tstop]
inp['Flpp']=[0,0,0]
inp['Flrst']=[720]
  
  #time interval to smooth the hydrodynamic boundary conditions
inp['Tlfsmo']=[0.]


  # set tide only run
#if 'tide' in kwargs.keys() :
#     if (kwargs['tide']==True) & (force==False):
#        inp['Sub1'] = ' '
#        inp['Filbnd']=basename+'.bnd'
#        inp['Filana']=basename+'.bca'
#       if 'Tidfor' not in order: order.append('Tidfor')
#       inp['Tidfor']=[['M2','S2','N2','K2'], \
#                      ['K1','O1','P1','Q1'], \
#                      ['-----------']]
 
  # specify ini file
# if 'Filic' not in order: order.append('Filic')
# inp['Filic']=basename+'.ini'

  # netCDF output
if 'FlNcdf' not in order: order.append('FlNcdf')

inp['FlNcdf'] = 'map his'


### create grid

In [11]:
#create grid
g=Grid() #initialize

In [12]:
#load model info
g.create(**b.__dict__)

In [13]:
# view grid
#g.__dict__

### extract meteo forcing

In [14]:
#set meteo forcing
m = ecmwf() # initialize

In [15]:
# specify the forecast times ft1,ft2
b.ft1 = 0
b.ft2 = 2 #that is 0,1 hours

In [16]:
# parse the ecmwf grib file.
m.parse(path='/Users/brey/Downloads/20130301.00.tropical_cyclone.grib',**b.__dict__)

100%|██████████| 6/6 [00:02<00:00,  2.45it/s]


meteo done


In [17]:
# view results
#m.__dict__

In [18]:
#load meteo to model
b.__dict__.update(m.__dict__)

In [19]:
#test
#b.__dict__

### extract bathymetry

In [20]:
#set dem data
d = gebco08(**b.__dict__) # initialize

In [21]:
d.get(file='/Volumes/BIG/StormS/BATHYMETRY/GLOBAL/gebco30_DELTARES.nc',grid_x=b.x,grid_y=b.y)

In [22]:
#load dem to model
b.__dict__.update(d.__dict__)

## output 

In [23]:
#create the model folder
path='/Users/brey/Downloads/test/'
if not os.path.exists(path):
    os.makedirs(path)

In [24]:
#save mdf file
mdf.write(inp, path+b.tag+'.mdf',selection=order)

In [25]:
g.properties = {'Coordinate System': 'Spherical', 'alfori': 0.0, 'xori': 0.0, 'yori': 0.0}

In [26]:
#save grid
g.write(path+b.tag+'.grd')

In [27]:
#write meteo out 
b.force(path=path)

In [28]:
# Write bathymetry file
ba = Dep()
# append the line/column of nodata 
nodata=np.empty(b.ni)
nodata.fill(np.nan)
bat1=np.vstack((b.idem,nodata))
nodata=np.empty((b.nj+1,1))
nodata.fill(np.nan)
bat2=np.hstack((bat1,nodata))
ba.val = bat2
ba.shape = bat2.shape

Dep.write(ba,path+b.tag+'.dep')

In [29]:
#write enc out
with open(path+b.tag+'.enc','w') as f:
    f.write('{:>5}{:>5}\n'.format(b.ni+1,1))  # add one like ddb
    f.write('{:>5}{:>5}\n'.format(b.ni+1,b.nj+1))
    f.write('{:>5}{:>5}\n'.format(1,b.nj+1))
    f.write('{:>5}{:>5}\n'.format(1,1))
    f.write('{:>5}{:>5}\n'.format(b.ni+1,1))

In [30]:
# save model for further use
b.save(path=path)

### execute

In [31]:
#execute
b.run(run_path=path,d3d='/Users/brey/DELFT3D',arch='lnx64',ncores=4)

MPI process number 000 has host unknown and is running on processor Ronin.local

--------------------------------------------------------------------------------

       Deltares, FLOW2D3D Version 6.02.10.7072, Mar 11 2017, 09:06:36

       libflow2d3d.so entry Flow2D3D::Run

--------------------------------------------------------------------------------



Part I    - Initialisation Time Dep. Data module...    

MPI process number 001 has host unknown and is running on processor Ronin.local

MPI process number 002 has host unknown and is running on processor Ronin.local

MPI process number 003 has host unknown and is running on processor Ronin.local

            runid : test

Part II   - Creating intermediate files...             

Part III  - Initialisation of the Execution module...  

Part IV   - Reading complete MD-file...                     

Part V    - Initialisation & checking input...              

Part VI   - Initialisation & checking second part...        

Part VII  - I